In [1]:
import pathlib
import pandas as pd
import numpy as np
import os

import h3

In [2]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import time

In [3]:
data=pd.read_excel('my_data.xlsx', sheet_name=0)


In [4]:
data=data.drop_duplicates(subset=['adres'])
data['longitude']=0
data['latitude']=0

data_adress=data[['adres','longitude','latitude']]
data_adress['adres']=data_adress['adres'].str.replace('\n', ' ')

data_adress

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,adres,longitude,latitude
0,"г. Миасс, ул. Силовая, 1",0,0
1,"1. Сосновский район, п. Есаульский, ул. Заго...",0,0
2,"1. г. Челябинск, в районе трамвайного кольца «...",0,0
3,"1. г. Челябинск, ул.Строительная, 21 (черный, ...",0,0
4,"г. Челябинск, ул. Автодорож ная, д. 10",0,0
...,...,...,...
57364,"197374, г. Санкт-Петербург, ул. Савушкина, д. ...",0,0
57365,"123022, г. Москва, ул. Красная Пресня, д. 31/2...",0,0
57366,"123022, г. Москва, ул. Красная Пресня, д. 31/2...",0,0
57367,"117997, г. Москва, ул. Вавилова, 19/454048, г....",0,0


In [5]:
from decimal import Decimal
from yandex_geocoder import Client
client = Client("key")


coordinates = client.coordinates('г. Москва, ул. Красная Пресня, д. 31/2')
coordinates[0]

Decimal('37.563323')

In [70]:
import threading
import time
import random
from random import randint

colll=6

def worker(i):
    time.sleep(randint(0, 20)/100)
    data_adress.loc[i,'longitude'] =1
    data_adress.loc[i,'latitude'] =1
    
    coordinates = client.coordinates(data_adress['adres'].loc[i])
    print(coordinates)
    data_adress.loc[i,'longitude'] =coordinates[0]
    data_adress.loc[i,'latitude'] =coordinates[1]

    #print(i)

for k in tqdm(range(int(len(data_adress[data_adress['longitude']==0])/colll))):

    time.sleep(0.05)
    
    for i in data_adress[data_adress['longitude']==0][:colll].index:
        t = threading.Thread(target=worker, args=(i,))
        t.start()
data_adress.to_csv('my_data_geo.csv') 

0it [00:00, ?it/s]


In [73]:
data_adress_cl=data_adress[data_adress['longitude']!=1]

In [74]:
data_adress_cl

,adres,longitude,latitude
0,"г. Миасс, ул. Силовая, 1",60.13998,55.020931
3,"1. г. Челябинск, ул.Строительная, 21 (черный, ...",61.642152,55.120841
4,"г. Челябинск, ул. Автодорож ная, д. 10",61.363764,55.208209
5,"1) г. Златоуст, ул. Уржумская, 75 2) г. Куса,...",59.409902,55.356222
6,"1. г. Златоуст, п. 6-ой жилой участок 2. г. Зл...",59.659813,55.17757
...,...,...,...
57364,"197374, г. Санкт-Петербург, ул. Савушкина, д. ...",30.291272,59.985845
57365,"123022, г. Москва, ул. Красная Пресня, д. 31/2...",37.574831,55.761365
57366,"123022, г. Москва, ул. Красная Пресня, д. 31/2...",61.401044,55.164176
57367,"117997, г. Москва, ул. Вавилова, 19/454048, г....",61.376323,55.156847


In [75]:
data_adress_cl.to_excel("data_adress_cl.xlsx",sheet_name='Sheet_name_1')  